In [4]:
# ============================================================
# 🚚 Proyecto: LogiPy
# Taller 2 – Programación y Decisiones
# Integrantes: Zara Arango y Nicolle Ospina
# ============================================================

# ============================================================
# 📦 Descripción del proyecto:
# LogiPy es un cotizador de envíos inteligente para pequeños e-commerce.
# Permite comparar costos entre transportadoras según distancia, peso y tarifas base,
# ayudando a elegir la opción más rentable.
# ============================================================


# ==========================
# 🔹 Importaciones
# ==========================
import sqlite3


# ==========================
# 🧍‍♀️ CLASE CLIENTE
# ==========================
class Cliente:
    """Representa un cliente que realiza envíos dentro del sistema LogiPy."""
    
    def __init__(self, nombre, cedula, direccion):
        self.nombre = nombre
        self.cedula = cedula
        self.direccion = direccion

    def __str__(self):
        return f"Cliente: {self.nombre} | C.C: {self.cedula} | Dirección: {self.direccion}"


# ==========================
# 📦 CLASE PAQUETE
# ==========================
class Paquete:
    """Contiene la información del paquete que será enviado."""
    
    def __init__(self, peso, largo, ancho, alto, destino):
        self.peso = peso
        self.largo = largo
        self.ancho = ancho
        self.alto = alto
        self.destino = destino

    def volumen(self):
        """Calcula el volumen del paquete en cm³."""
        return self.largo * self.ancho * self.alto

    def __str__(self):
        return f"Paquete: {self.peso}kg | Volumen: {self.volumen()}cm³ | Destino: {self.destino}"


# ==========================
# 🚛 CLASE BASE TRANSPORTADORA
# ==========================
class Transportadora:
    """Clase base para las transportadoras del sistema LogiPy."""
    
    def __init__(self, nombre, tarifa_base, costo_por_km):
        self.nombre = nombre
        self.tarifa_base = tarifa_base
        self.costo_por_km = costo_por_km

    def calcular_envio(self, distancia, peso):
        """Calcula el costo total del envío según distancia y peso."""
        return self.tarifa_base + (self.costo_por_km * distancia) + (peso * 1000)

    def resumen(self):
        """Devuelve un resumen con la información de la transportadora."""
        return f"Transportadora: {self.nombre} | Tarifa base: ${self.tarifa_base} | Costo por km: ${self.costo_por_km}"


# ==========================
# 🟢 SUBCLASES DE TRANSPORTADORA
# ==========================
class Servientrega(Transportadora):
    """Transportadora Servientrega con descuento para paquetes livianos."""
    
    def calcular_envio(self, distancia, peso):
        costo = super().calcular_envio(distancia, peso)
        # 10% de descuento si el peso es menor de 2 kg
        if peso < 2:
            costo *= 0.9
        return costo


class Interrapidisimo(Transportadora):
    """Transportadora Interrapidisimo con recargo por rapidez."""
    
    def calcular_envio(self, distancia, peso):
        costo = super().calcular_envio(distancia, peso)
        # 5% adicional por servicio rápido
        return costo * 1.05


# ==========================
# ✉️ CLASE ENVÍO
# ==========================
class Envio:
    """Asocia un cliente, paquete y transportadora para calcular el costo total del envío."""
    
    def __init__(self, cliente, paquete, transportadora, distancia):
        self.cliente = cliente
        self.paquete = paquete
        self.transportadora = transportadora
        self.distancia = distancia
        self.costo_total = 0

    def calcular_costo(self):
        """Calcula el costo del envío usando la transportadora seleccionada."""
        self.costo_total = self.transportadora.calcular_envio(self.distancia, self.paquete.peso)
        return self.costo_total

    def resumen_envio(self):
        """Muestra un resumen del envío con cliente, destino, transportadora y costo."""
        return (f"{self.cliente.nombre} enviará un paquete a {self.paquete.destino} "
                f"con {self.transportadora.nombre}. Costo total: ${self.costo_total:.2f}")


# ==========================
# 🗄️ FUNCIONES DE BASE DE DATOS
# ==========================
def crear_bd():
    """Crea la base de datos y la tabla de transportadoras si no existen."""
    conexion = sqlite3.connect("logipy.db")
    cursor = conexion.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Transportadora (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nombre TEXT NOT NULL,
        tarifa_base REAL NOT NULL,
        costo_por_km REAL NOT NULL
    );
    """)

    conexion.commit()
    conexion.close()


def insertar_transportadoras():
    """Inserta transportadoras predeterminadas en la base de datos."""
    conexion = sqlite3.connect("logipy.db")
    cursor = conexion.cursor()

    cursor.execute("INSERT INTO Transportadora (nombre, tarifa_base, costo_por_km) VALUES (?, ?, ?)", 
                   ("Servientrega", 5000, 200))
    cursor.execute("INSERT INTO Transportadora (nombre, tarifa_base, costo_por_km) VALUES (?, ?, ?)", 
                   ("Interrapidisimo", 4500, 250))

    conexion.commit()
    conexion.close()


def mostrar_transportadoras():
    """Muestra todas las transportadoras registradas en la base de datos."""
    conexion = sqlite3.connect("logipy.db")
    cursor = conexion.cursor()
    cursor.execute("SELECT * FROM Transportadora")
    for fila in cursor.fetchall():
        print(fila)
    conexion.close()


# ==========================
# 🧪 BLOQUE DE PRUEBA
# ==========================
if __name__ == "__main__":
    # Crear base de datos y registrar transportadoras
    crear_bd()
    insertar_transportadoras()

    # Crear objetos de ejemplo
    cliente1 = Cliente("Zara Arango", "1009876543", "Bogotá")
    paquete1 = Paquete(1.5, 30, 20, 15, "Medellín")
    servientrega = Servientrega("Servientrega", 5000, 200)

    # Crear y calcular envío
    envio1 = Envio(cliente1, paquete1, servientrega, distancia=400)
    costo = envio1.calcular_costo()

    # Mostrar resultados
    print(cliente1)
    print(paquete1)
    print(servientrega.resumen())
    print(envio1.resumen_envio())

    # Mostrar base de datos
    print("\nTransportadoras registradas en la base de datos:")
    mostrar_transportadoras()

Cliente: Zara Arango | C.C: 1009876543 | Dirección: Bogotá
Paquete: 1.5kg | Volumen: 9000cm³ | Destino: Medellín
Transportadora: Servientrega | Tarifa base: $5000 | Costo por km: $200
Zara Arango enviará un paquete a Medellín con Servientrega. Costo total: $77850.00

Transportadoras registradas en la base de datos:
(1, 'Servientrega', 5000.0, 200.0)
(2, 'Interrapidísimo', 4500.0, 250.0)
(3, 'Servientrega', 5000.0, 200.0)
(4, 'Interrapidisimo', 4500.0, 250.0)


In [5]:
# ============================================================
# ✅ Paso 3: Pruebas del prototipo LogiPy
# ============================================================

# Casos de prueba de envíos con diferentes transportadoras

# Crear clientes
cliente1 = Cliente("Zara Arango", "1009876543", "Bogotá")
cliente2 = Cliente("Nicolle Ospina", "1092345678", "Cali")

# Crear paquetes
paquete1 = Paquete(1.2, 20, 15, 10, "Medellín")
paquete2 = Paquete(5.0, 50, 40, 30, "Barranquilla")

# Crear transportadoras
servientrega = Servientrega("Servientrega", 5000, 200)
interrapidisimo = Interrapidisimo("Interrapidisimo", 4500, 250)

# Crear envíos y calcular costos
envio1 = Envio(cliente1, paquete1, servientrega, distancia=400)
envio2 = Envio(cliente2, paquete2, interrapidisimo, distancia=800)

# Mostrar resultados
print("=== RESULTADOS DE PRUEBA ===")
print(envio1.resumen_envio())
print(envio2.resumen_envio())

# Validar almacenamiento en base de datos
print("\n=== TRANSPORTADORAS EN BD ===")
mostrar_transportadoras()

=== RESULTADOS DE PRUEBA ===
Zara Arango enviará un paquete a Medellín con Servientrega. Costo total: $0.00
Nicolle Ospina enviará un paquete a Barranquilla con Interrapidisimo. Costo total: $0.00

=== TRANSPORTADORAS EN BD ===
(1, 'Servientrega', 5000.0, 200.0)
(2, 'Interrapidísimo', 4500.0, 250.0)
(3, 'Servientrega', 5000.0, 200.0)
(4, 'Interrapidisimo', 4500.0, 250.0)


In [4]:
# ===========================
# Paso 4 - Celda auto-contenida: guardar/envíos en SQLite
# Ejecuta esta celda sola (no requiere que `envio1` exista)
# ===========================
import sqlite3

# --- 1) Asegurar que las clases existan (si no, definir versiones mínimas)
if 'Cliente' not in globals():
    class Cliente:
        def __init__(self, nombre, cedula, direccion):
            self.nombre = nombre
            self.cedula = cedula
            self.direccion = direccion

if 'Paquete' not in globals():
    class Paquete:
        def __init__(self, peso, largo, ancho, alto, destino):
            self.peso = peso
            self.largo = largo
            self.ancho = ancho
            self.alto = alto
            self.destino = destino
        def volumen(self):
            return self.largo * self.ancho * self.alto

if 'Transportadora' not in globals():
    class Transportadora:
        def __init__(self, nombre, tarifa_base, costo_por_km):
            self.nombre = nombre
            self.tarifa_base = tarifa_base
            self.costo_por_km = costo_por_km
        def calcular_envio(self, distancia, peso):
            return self.tarifa_base + (self.costo_por_km * distancia) + (peso * 1000)
        def resumen(self):
            return f"Transportadora: {self.nombre} | Tarifa base: {self.tarifa_base} | Costo/km: {self.costo_por_km}"

if 'Servientrega' not in globals():
    class Servientrega(Transportadora):
        def calcular_envio(self, distancia, peso):
            costo = super().calcular_envio(distancia, peso)
            if peso < 2:
                costo *= 0.9
            return costo

if 'Interrapidisimo' not in globals():
    class Interrapidisimo(Transportadora):
        def calcular_envio(self, distancia, peso):
            costo = super().calcular_envio(distancia, peso)
            return costo * 1.05

if 'Envio' not in globals():
    class Envio:
        def __init__(self, cliente, paquete, transportadora, distancia):
            self.cliente = cliente
            self.paquete = paquete
            self.transportadora = transportadora
            self.distancia = distancia
            self.costo_total = 0
        def calcular_costo(self):
            self.costo_total = self.transportadora.calcular_envio(self.distancia, self.paquete.peso)
            return self.costo_total
        def resumen_envio(self):
            return f"{self.cliente.nombre} -> {self.paquete.destino} via {self.transportadora.nombre} | ${self.costo_total:.2f}"

# --- 2) Funciones DB
def crear_tabla_envios():
    conexion = sqlite3.connect("logipy.db")
    cursor = conexion.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Envio (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        cliente TEXT NOT NULL,
        cedula TEXT NOT NULL,
        direccion TEXT NOT NULL,
        destino TEXT NOT NULL,
        transportadora TEXT NOT NULL,
        distancia REAL NOT NULL,
        costo_total REAL NOT NULL
    );
    """)
    conexion.commit()
    conexion.close()

def guardar_envio(envio):
    # calcula costo si no está calculado
    if getattr(envio, "costo_total", 0) == 0:
        envio.calcular_costo()
    conexion = sqlite3.connect("logipy.db")
    cursor = conexion.cursor()
    # Evitar duplicados: buscar por los campos clave
    cursor.execute("""
        SELECT id FROM Envio WHERE cliente=? AND destino=? AND transportadora=? AND distancia=? AND costo_total=?
    """, (envio.cliente.nombre, envio.paquete.destino, envio.transportadora.nombre, envio.distancia, envio.costo_total))
    if cursor.fetchone() is None:
        cursor.execute("""
            INSERT INTO Envio (cliente, cedula, direccion, destino, transportadora, distancia, costo_total)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (
            envio.cliente.nombre,
            envio.cliente.cedula,
            envio.cliente.direccion,
            envio.paquete.destino,
            envio.transportadora.nombre,
            envio.distancia,
            envio.costo_total
        ))
        conexion.commit()
        print(f"[DB] Envío guardado: {envio.resumen_envio()}")
    else:
        print(f"[DB] Saltado (ya existe): {envio.cliente.nombre} -> {envio.paquete.destino} via {envio.transportadora.nombre}")
    conexion.close()

def mostrar_envios():
    conexion = sqlite3.connect("logipy.db")
    cursor = conexion.cursor()
    cursor.execute("SELECT id, cliente, destino, transportadora, distancia, costo_total FROM Envio ORDER BY id")
    filas = cursor.fetchall()
    if not filas:
        print("No hay envíos registrados.")
    else:
        print("\\n=== ENVÍOS REGISTRADOS EN DB ===")
        for f in filas:
            print(f)
    conexion.close()

# --- 3) Preparar envíos a guardar: preferir 'envio1/envio2' si existen
envios_a_guardar = []

if 'envio1' in globals():
    try:
        envio1.calcular_costo()
    except Exception:
        pass
    envios_a_guardar.append(envio1)

if 'envio2' in globals():
    try:
        envio2.calcular_costo()
    except Exception:
        pass
    envios_a_guardar.append(envio2)

# Si no hay envíos en memoria, crear ejemplos (no rompe tu código)
if not envios_a_guardar:
    c1 = Cliente("Zara Arango", "1009876543", "Bogotá")
    p1 = Paquete(1.5, 30, 20, 15, "Medellín")
    s1 = Servientrega("Servientrega", 5000, 200)
    e1 = Envio(c1, p1, s1, distancia=400)
    e1.calcular_costo()
    envios_a_guardar.append(e1)

    c2 = Cliente("Nicolle Ospina", "1092345678", "Cali")
    p2 = Paquete(5.0, 50, 40, 30, "Barranquilla")
    i2 = Interrapidisimo("Interrapidisimo", 4500, 250)
    e2 = Envio(c2, p2, i2, distancia=800)
    e2.calcular_costo()
    envios_a_guardar.append(e2)

# --- 4) Ejecutar guardado y mostrar
crear_tabla_envios()
for e in envios_a_guardar:
    guardar_envio(e)

mostrar_envios()

[DB] Envío guardado: Zara Arango -> Medellín via Servientrega | $77850.00
[DB] Envío guardado: Nicolle Ospina -> Barranquilla via Interrapidisimo | $219975.00
\n=== ENVÍOS REGISTRADOS EN DB ===
(1, 'Zara Arango', 'Medellín', 'Servientrega', 400.0, 77850.0)
(2, 'Nicolle Ospina', 'Barranquilla', 'Interrapidisimo', 800.0, 219975.0)
